# Daily covid-19 Deaths compared to average deaths the last 10 years
> Awesome summary
- toc:true- branch: master
- badges: true
- comments: true
- author: Lode Nachtergaele
- categories: [cast42, jupyter, covid19, Belgium, altair]

In [0]:
# Import pandas for data wrangling and Altair for plotting
import pandas as pd
import altair as alt

The number of deadths per day from 2008 until 2018 can obtained from Statbel:


In [0]:
df = pd.read_excel('https://statbel.fgov.be/sites/default/files/files/opendata/bevolking/TF_DEATHS.xlsx') # , skiprows=5, sheet_name=sheetnames

In [3]:
# Get a quick look to the data
df.head()

,DT_DATE,MS_NUM_DEATHS
0,2008-01-01,342
1,2008-01-02,348
2,2008-01-03,340
3,2008-01-04,349
4,2008-01-05,348


In [0]:
df['Jaar'] = df['DT_DATE'].dt.year
df['Dag'] = df['DT_DATE'].dt.dayofyear

In [0]:
df_plot = df.groupby('Dag')['MS_NUM_DEATHS'].mean().to_frame().reset_index()

In [6]:
# Let's make a quick plot
alt.Chart(df_plot).mark_line().encode(x='Dag', y='MS_NUM_DEATHS').properties(width=600)

alt.Chart(...)

The [John Hopkings University CSSE](https://systems.jhu.edu/research/public-health/ncov/) keeps track of the number of covid-19 deadths per day and county on the a github reposotry: [https://github.com/CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19)

In [0]:
# Obtain the data
deaths_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths = pd.read_csv(deaths_url, sep=',')

In [0]:
# Filter out Belgium
deaths_be = deaths[deaths['Country/Region'] == 'Belgium']

In [9]:
# Inspect how the data is stored
deaths_be

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20
23,NaN,Belgium,50.8333,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,3,3,3,4,4,5,10,14,21,37,67,75,88,122,178,220,289,353,431,513,705,828,1011,1143,1283,1447,1632,2035,2240,2523,3019,3346,3600,3903,4157,4440,4857,5163


In [0]:
# Create dateframe for plotting
df_deaths = pd.DataFrame(data={'Datum':pd.to_datetime(deaths_be.columns[4:]), 'Overlijdens':deaths_be.iloc[0].values[4:]})

In [11]:
# Check for Nan's
df_deaths['Overlijdens'].isna().sum()

0

In [0]:
# Convert to integer
df_deaths['Overlijdens'] = df_deaths['Overlijdens'].astype(int)

In [0]:
# Add day of the year to the dataframe
df_deaths['Dag'] = df_deaths['Datum'].dt.dayofyear

In [0]:
# Plot the data
dead_2008_2018 = alt.Chart(df_plot).mark_line().encode(x='Dag', y='MS_NUM_DEATHS')

In [0]:
# Calculate the day-by-day change
df_deaths['Nieuwe covid-19 Sterfgevallen'] = df_deaths['Overlijdens'].diff()

In [16]:
# Check types
df_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Datum                          87 non-null     datetime64[ns]
 1   Overlijdens                    87 non-null     int64         
 2   Dag                            87 non-null     int64         
 3   Nieuwe covid-19 Sterfgevallen  86 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 2.8 KB


In [17]:
# Plot covid-19 deaths in Belgium according to JHU CSSE
dead_covid= alt.Chart(df_deaths).mark_line(point=True).encode(x=alt.X('Dag',scale=alt.Scale(domain=(1, 110), clamp=True)),
 y='Nieuwe covid-19 Sterfgevallen', color=alt.ColorValue('red'), tooltip=['Dag', 'Nieuwe covid-19 Sterfgevallen'])
dead_covid

alt.Chart(...)

In [18]:
# add average deaths per day in the last 10 year
dead_2008_2018 + dead_covid

alt.LayerChart(...)

In [19]:
df.head()

,DT_DATE,MS_NUM_DEATHS,Jaar,Dag
0,2008-01-01,342,2008,1
1,2008-01-02,348,2008,2
2,2008-01-03,340,2008,3
3,2008-01-04,349,2008,4
4,2008-01-05,348,2008,5


In [0]:
df['Datum'] = pd.to_datetime(df['DT_DATE'])

In [21]:
line = alt.Chart(df).mark_line().encode(
    x=alt.X('Dag', scale=alt.Scale(
            domain=(1, 120),
            clamp=True
        )),
    y='mean(MS_NUM_DEATHS)'
)

# Bootstrapped 95% confidence interval
band = alt.Chart(df).mark_errorband(extent='ci').encode(
    x=alt.X('Dag', scale=alt.Scale(domain=(1, 120), clamp=True)),
    y=alt.Y('MS_NUM_DEATHS', title='Overlijdens per dag'),
)

dead_covid= alt.Chart(df_deaths).mark_line(point=True).encode(
    x=alt.X('Dag',scale=alt.Scale(domain=(1, 120), clamp=True)),
    y='Nieuwe covid-19 Sterfgevallen',
    color=alt.ColorValue('red'),
    tooltip=['Dag', 'Nieuwe covid-19 Sterfgevallen', 'Datum']
)

(band + line + dead_covid).properties(width=1024, title='Gemiddeld aantal overlijdens over 10 jaar versus overlijdens door covid-19 in Belgie')

alt.LayerChart(...)

# Source date from sciensano

In [0]:
df_sc = pd.read_csv('https://epistat.sciensano.be/Data/COVID19BE_MORT.csv')

In [23]:
df_sc.head()

,DATE,REGION,AGEGROUP,SEX,DEATHS
0,2020-03-10,Brussels,85+,F,1
1,2020-03-11,Flanders,85+,F,1
2,2020-03-11,Brussels,75-84,M,1
3,2020-03-11,Brussels,85+,F,1
4,2020-03-12,Brussels,75-84,M,1


In [0]:
df_dead_day = df_sc.groupby('DATE')['DEATHS'].sum().reset_index()
df_dead_day['Datum'] = pd.to_datetime(df_dead_day['DATE'])
df_dead_day['Dag'] = df_dead_day['Datum'].dt.dayofyear


In [25]:
line = alt.Chart(df).mark_line().encode(
    x=alt.X('Dag', scale=alt.Scale(
            domain=(1, 120),
            clamp=True
        )),
    y='mean(MS_NUM_DEATHS)'
)

# Bootstrapped 95% confidence interval
band = alt.Chart(df).mark_errorband(extent='ci').encode(
    x=alt.X('Dag', scale=alt.Scale(domain=(1, 120), clamp=True)),
    y=alt.Y('MS_NUM_DEATHS', title='Overlijdens per dag'),
)

dead_covid= alt.Chart(df_dead_day).mark_line(point=True).encode(
    x=alt.X('Dag',scale=alt.Scale(domain=(1, 120), clamp=True)),
    y='DEATHS',
    color=alt.ColorValue('red'),
    tooltip=['Dag', 'DEATHS', 'Datum']
)

(band + line + dead_covid).properties(width=1024, title='Gemiddeld aantal overlijdens over 10 jaar versus overlijdens door covid-19 in Belgie')

alt.LayerChart(...)

# Obtain more detail (for another blogpost...)

In [0]:
df_tot_sc = pd.read_excel('https://epistat.sciensano.be/Data/COVID19BE.xlsx')

In [27]:
df_tot_sc


,DATE,PROVINCE,REGION,AGEGROUP,SEX,CASES
0,2020-03-01,Brussels,Brussels,10-19,M,1
1,2020-03-01,Brussels,Brussels,10-19,F,1
2,2020-03-01,Brussels,Brussels,20-29,M,1
3,2020-03-01,Brussels,Brussels,30-39,F,1
4,2020-03-01,Brussels,Brussels,40-49,F,1
...,...,...,...,...,...,...
6693,NaN,OostVlaanderen,Flanders,NaN,F,4
6694,NaN,VlaamsBrabant,Flanders,40-49,M,3
6695,NaN,VlaamsBrabant,Flanders,40-49,F,2
6696,NaN,VlaamsBrabant,Flanders,50-59,M,1
